In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
import os
os.environ['OPENAI_API_KEY']='sk-or-v1-'
os.environ['OPENAI_API_BASE']="https://openrouter.ai/api/v1"

In [3]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [ ]:
#creating tools


@tool
def get_conversion_factor(base_currency:str,target_currency:str)->float:
    """This function fetch the currency conversion factor between then base currency and the target currency"""
    url=f'https://v6.exchangerate-api.com/v6/96f/pair/{base_currency}/{target_currency}'

    response=requests.get(url)
    return response.json()

@tool 
def convert(base_currency:float,conversion_rate:Annotated[float,InjectedToolArg])->float: # with this injected tool arg LLM wont set the value by itself
    """Given a conversion rate this function will calculate the target current value from  base curreny value"""

    return base_currency*conversion_rate


In [5]:
result=get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

In [6]:
result['conversion_rate']

86.6689

In [7]:
convert.invoke({'base_currency':10,'conversion_rate':86.6689})

866.689

### tool binding

In [8]:
llm=ChatOpenAI()

In [9]:
llm_with_tools=llm.bind_tools([get_conversion_factor,convert])

In [10]:
messages=[HumanMessage('What is the converstion factor between USD and INR , and based on that can you convert 10 usd to INR')]

In [11]:
messages

[HumanMessage(content='What is the converstion factor between USD and INR , and based on that can you convert 10 usd to INR', additional_kwargs={}, response_metadata={})]

In [12]:
ai_message=llm_with_tools.invoke(messages)

In [13]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_p5HH8ZunKpxUAPS6cqWQcCe6',
  'type': 'tool_call'}]

### now our logic failed why because llm took our both question sequentially , to get the converstion factor, and to convert so in convert tool call it got some conversion from it's train data it didn't get the conversion factor from the our function

## now to solve that we use Injected Argument Method

In [14]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_p5HH8ZunKpxUAPS6cqWQcCe6',
  'type': 'tool_call'}]

In [15]:
messages.append(ai_message)

In [16]:
messages

[HumanMessage(content='What is the converstion factor between USD and INR , and based on that can you convert 10 usd to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p5HH8ZunKpxUAPS6cqWQcCe6', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 123, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'openai/gpt-3.5-turbo', 'system_fingerprint': None, 'id': 'gen-1750694069-DjkZG3nSowQufBi3ZDEA', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d6b3d7b2-0d03-45b9-9474-2d3c821cac14-0', tool_calls=[{'n

In [17]:
import json # the toolmessage output is in json we need to convert it
for tool_call in ai_message.tool_calls:
    #execute the 1st tool and get the conversion rate
    if tool_call['name']=='get_conversion_factor':
        toolmessage1=get_conversion_factor.invoke(tool_call)
        #fetch this conversion rate
        convertion_rate=json.loads(toolmessage1.content)['conversion_rate']
        #append the tool message to messages list
        messages.append(toolmessage1)
    #execute the 2nd tool using the conversion rate from tool1

    if tool_call['name']=='convert':
        #fetch the current argument
        tool_call['args']['conversion_rate']=convertion_rate
        tool_message2=convert.invoke(tool_call)
        messages.append(tool_message2)


In [18]:
messages

[HumanMessage(content='What is the converstion factor between USD and INR , and based on that can you convert 10 usd to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p5HH8ZunKpxUAPS6cqWQcCe6', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 123, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'openai/gpt-3.5-turbo', 'system_fingerprint': None, 'id': 'gen-1750694069-DjkZG3nSowQufBi3ZDEA', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d6b3d7b2-0d03-45b9-9474-2d3c821cac14-0', tool_calls=[{'n

In [19]:
llm_with_tools.invoke(messages).content

''

In [20]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\avani\AppData\Local\Temp\ipykernel_23024\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [22]:
# --- Step 6: Run the Agent ---
user_query = "Convert 1000 INR to USD"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: I need to convert 1000 INR to USD. I can do this by using the currency conversion tool.

Action:
```
{
  "action": "get_conversion_factor",
  "action_input": {"base_currency": "INR", "target_currency": "USD"}
}
```
Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1750636801, 'time_last_update_utc': 'Mon, 23 Jun 2025 00:00:01 +0000', 'time_next_update_unix': 1750723201, 'time_next_update_utc': 'Tue, 24 Jun 2025 00:00:01 +0000', 'base_code': 'INR', 'target_code': 'USD', 'conversion_rate': 0.01154}
Thought:Action:
```
{
  "action": "convert",
  "action_input": {"base_currency": 1000, "conversion_rate": 0.01154}
}
``` 

Observation: 11.54
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "1000 INR is equal to 11.54 USD"
}
``` 


> Finished chain.
